# İndirilen modüller 
Alınan ses dosyalarını oynatmak (pysoundfile,bitstring) ve konuşmaki gürültü leri temizlemek (noisereduce) için modüller indirildi. 



In [ ]:
!pip install pysoundfile
!pip install bitstring

     |████████████████████████████████| 204kB 6.8MB/s 
  Created wheel for bitstring: filename=bitstring-3.1.7-cp37-none-any.whl size=37949 sha256=703be8ad78d82525f0edb1be97bcdb6936fc5b5267308c9b65a8e13200659125
  Stored in directory: /root/.cache/pip/wheels/b8/27/f0/8373e26b7de57db03dc18aaaebdd8c26a99da882416f762979
Successfully built bitstring


In [1]:
!pip install noisereduce

  Created wheel for noisereduce: filename=noisereduce-1.1.0-cp37-none-any.whl size=7610 sha256=392f281c901cae3cd2fcc15c4f5fe96bd79be64f2c0df44dd880f3481f5a13f3
  Stored in directory: /root/.cache/pip/wheels/a6/2c/70/f9ccb41280dcfbe3eaeb7930f913dd85394617f3d3136f29cc
Successfully built noisereduce


# Giriş
Bu kısımda kayedilen TESS modeli ve uygulama için toplanan ses verilerin okunması yer almaktadır.

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from keras.models import load_model
model_path = r"/content/drive/MyDrive/Bitirme/Uygulama/Save_Model/TESS/cnn1d.h5"  # kaydedilen modeliin dizin yolu.
model = load_model(model_path)

## Önişlem fonksiyonu 
Ortak olan önişlemler için yazılan fonksiyon


1.   Gürültü temizleme
2.   Sınıflandırma uygun bölünmesi



In [17]:
def preprocessing_data(data):
    # Gürültü temizleme
    noisy_part = data[0:15000]
    data = nr.reduce_noise(audio_clip=out,noise_clip=noisy_part)
    played_clean_data.append(data) # Temizlenmiş sesleri oynatmak için listeye alındı.
    # CNN sabit uzunlukta çıktı aldığından sesin bloklara bölünmesi
    max_length = 30600
    iter = int((len(data)-(len(data)%max_length))/max_length)
    arr = []
    for j in range(iter):
        arr.append(data[(j*max_length):((j+1)*max_length)])
    return arr

## Duygu tahmin etme fonksiyonu 

In [5]:
def predict_emotion(data):
    trans_emo = {0:'kızgın',1:'iğrenme',2:'korku',3:'mutlu',4:'doğal',5:'şaşkın',6:'üzgün'}
    emo = []
    emo_ist = []
    arr = []
    for i in data:
        example = lb.feature.mfcc(y=i,n_mfcc=20).reshape(1,1200,1) # 20 adet mfcc özniteliği çıkarılıp yeniden boyutlandırıldı. 
        my_list = model.predict(example)[0]
        maxval = max(my_list)
        indices = [index for index, val in enumerate(my_list) if val == maxval]
        emo.append(trans_emo[int(indices[0])])
        emo_ist.append(maxval)
        #arr.append(trans_emo[indices[0]])
        arr.append([trans_emo[int(indices[0])],maxval])
    maxval = max(emo_ist)
    indices = [index for index, val in enumerate(emo_ist) if val == maxval]
    print("Tahmin Edilen Duygu:",emo[indices[0]])
    print("Tahmin Sonucu:",maxval)

In [22]:
import numpy as np
import librosa as lb
import os
import noisereduce as nr
predict_path = r"/content/drive/MyDrive/Bitirme/Uygulama/Datasets/predict_data" # ses verilerin dizi yolu
predict_data = []
predict_labels = []
played_data = []
played_clean_data = []
for filename in os.listdir(predict_path):
    data_path = os.path.join(predict_path, filename)
    out, sr= lb.load(data_path,sr=None)
    played_data.append(out)  # Orjinal sesleri oynatmak için listeye alındı
    data = preprocessing_data(out)
    predict_data.append(data)
    predict_labels.append(filename.split('.')[0])

## Modelin sınıflandırma sonuçları

In [23]:
sayac = 0
for i in predict_data:
    print("="*100)
    print(sayac,"Duygu Etiketi:",predict_labels[sayac].split("-")[0])
    print("Konuşma İfadesi:",predict_labels[sayac].split("-")[1])
    predict_value = predict_emotion(i)
    sayac +=1

0 Duygu Etiketi: korku
Konuşma İfadesi: No_dont_touch_me
Tahmin Edilen Duygu: üzgün
Tahmin Sonucu: 0.7114222
1 Duygu Etiketi: üzgün
Konuşma İfadesi: Be_happy_Mel
Tahmin Edilen Duygu: üzgün
Tahmin Sonucu: 0.97265697
2 Duygu Etiketi: üzgün
Konuşma İfadesi: Im_not_like_you
Tahmin Edilen Duygu: üzgün
Tahmin Sonucu: 0.79821855
3 Duygu Etiketi: doğal
Konuşma İfadesi: Good_morning_everybody
Tahmin Edilen Duygu: doğal
Tahmin Sonucu: 0.7206097


# Gürültü temizleme örneği
1. Bu kısımda ses verilerinde yapılan temizleme işlemine verilen örneğin incelenmesini adına yazıldı. 
2. Played_data ve played_clean_data listelerinin indisleri değiştirilerek diğer ses verilerine bakılabilir. 

In [9]:
import IPython
IPython.display.Audio(data=played_data[3],rate=sr,autoplay=False)

In [10]:
IPython.display.Audio(data=played_clean_data[3],rate=sr,autoplay=False)

#  Uygulama II
Bu kısımda ana dili Türkçe olan İngilizce Öğretmenin ses verilerin testi yapıldı.

In [18]:
predict_path = r"/content/drive/MyDrive/Bitirme/Uygulama/Datasets/predict_data(Ingilize_Ogretmeni)"
predict_data = []
predict_labels = []
played_data = []
played_clean_data = []
for filename in os.listdir(predict_path):
    data_path = os.path.join(predict_path, filename)
    out, sr= lb.load(data_path,sr=None)
    played_data.append(out)
    data = preprocessing_data(out)
    predict_data.append(data)
    predict_labels.append(filename.split('.')[0])

In [19]:
sayac = 0
for i in predict_data:
    print("="*100)
    print(sayac,"Duygu Etiketi:",predict_labels[sayac].split("-")[0])
    print("Konuşma İfadesi:",predict_labels[sayac].split("-")[1])
    predict_value = predict_emotion(i)
    sayac +=1

0 Duygu Etiketi: üzgün
Konuşma İfadesi: Im_really_sorry_forgive_me
Tahmin Edilen Duygu: doğal
Tahmin Sonucu: 0.82176393
1 Duygu Etiketi: mutlu
Konuşma İfadesi: Idont understand anything you say and at the same time Im very glad to be here
Tahmin Edilen Duygu: iğrenme
Tahmin Sonucu: 0.889169
2 Duygu Etiketi: şaşkın
Konuşma İfadesi: This is great Might be the most fascinating ring Ive ever seen
Tahmin Edilen Duygu: doğal
Tahmin Sonucu: 0.991578
3 Duygu Etiketi: korku
Konuşma İfadesi: Seanyou scared me what do you want
Tahmin Edilen Duygu: doğal
Tahmin Sonucu: 0.9090746
4 Duygu Etiketi: iğrenme
Konuşma İfadesi: Ew(ıyyy)this oven is disgusting sorry
Tahmin Edilen Duygu: üzgün
Tahmin Sonucu: 0.9092073
5 Duygu Etiketi: kızgın
Konuşma İfadesi: If youre going to TALK About Me BehindMYBackDontSmileAtMeToMyFace
Tahmin Edilen Duygu: doğal
Tahmin Sonucu: 0.833987
6 Duygu Etiketi: doğal
Konuşma İfadesi: Enjoy the little things in life_because one day you will look back and realize they were t

In [20]:
IPython.display.Audio(data=played_data[3],rate=sr,autoplay=False)

In [21]:
IPython.display.Audio(data=played_clean_data[3],rate=sr,autoplay=False)